In [1]:
import os
os.chdir("/home/hxcai/cell_type_specific_CRE/data/ATAC_seq")

import pandas as pd
import subprocess
import numpy as np

In [2]:
def process_data(input_file_name, output_file_name, label):
    """
    file_name should be 'xxx.bed'
    """
    df = pd.read_csv(input_file_name, sep='\t', header=None)
    df = df[[0,1,2]]
    df[3] = label
    df = df.sort_values(by=[0,1,2])
    df.to_csv(output_file_name, sep='\t', header=None, index=None)

In [3]:
def intersect_data(input_file_name, output_file_name):
    cmd = f'bedtools intersect -wa -f 0.5 -c -a allTFs.pos.bed -b {input_file_name} > {output_file_name} \n'
    subprocess.call(cmd, shell=True)
    
    data = pd.read_csv(output_file_name, sep='\t', header=None)
    data[6] = data[6].astype(bool).astype(int)
    data = data[[0, 1, 2, 6]]
    data.to_csv(output_file_name, sep='\t', index=False, header=False)

In [4]:
def aug_positive_samples(input_file_name, output_file_name, shifts=[20, 40, 60, 80, 100, 120, 140, 160, 180]):
    data = pd.read_csv(input_file_name, sep='\t', header=None)

    df_expanded = pd.DataFrame()
    for shift in shifts:
        df_positive = data[data.iloc[:, 3] == 1].copy()
        df_positive.iloc[:, 1] += shift
        df_positive.iloc[:, 2] += shift
        df_expanded = pd.concat([df_expanded, df_positive])

    df_combined = pd.concat([data, df_expanded], ignore_index=True).sort_values(by=[0,1,2])
    df_combined.to_csv(output_file_name, sep='\t', header=None, index=None)

In [5]:
process_data('HepG2_ENCFF913MQB.bed', 'sorted_HepG2.bed', 'HepG2_ATAC_Peak')
intersect_data('sorted_HepG2.bed', 'sorted_HepG2_intersect.bed')
aug_positive_samples('sorted_HepG2_intersect.bed', 'auged_sorted_HepG2_intersect.bed')

In [11]:
process_data('GM12878_CTCF.bed', 'sorted_GM12878_CTCF.bed', '1')
intersect_data('sorted_GM12878_CTCF.bed', 'sorted_GM12878_CTCF_intersect.bed')
# aug_positive_samples('sorted_GM12878_CTCF_intersect.bed', 'auged_sorted_GM12878_CTCF_intersect.bed', 5*np.arange(1, 40))
aug_positive_samples('sorted_GM12878_CTCF_intersect.bed', 'auged_sorted_GM12878_CTCF_intersect.bed', [0]*20)

In [8]:
df = pd.read_csv('sorted_HepG2_intersect.bed', sep='\t', header=None)
print(np.sum(df[3] == 1) / len(df))

df = pd.read_csv('auged_sorted_HepG2_intersect.bed', sep='\t', header=None)
print(np.sum(df[3] == 1) / len(df))

0.036296546790062965
0.2735915963093399


In [12]:
df = pd.read_csv('auged_sorted_GM12878_CTCF_intersect.bed', sep='\t', header=None)
print(np.sum(df[3] == 1) / len(df))

0.3915553485439459
